In [3]:
from google.colab import drive
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
import gensim.downloader as api
import keras

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
drive.mount('/content/drive')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[==================================================] 100.0% 199.5/199.5MB downloaded


In [ ]:
from __future__ import print_function
from keras.callbacks import LambdaCallback, ModelCheckpoint, EarlyStopping
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, LSTM, Bidirectional, Embedding
import numpy as np
import sys
import io
import os

# Parameters: change to experiment different configurations
SEQUENCE_LEN = 10
MIN_WORD_FREQUENCY = 10
STEP = 1
BATCH_SIZE = 32


def shuffle_and_split_training_set(sentences_original, next_original, percentage_test=2):
    # shuffle at unison
    print('Shuffling sentences')

    tmp_sentences = []
    tmp_next_word = []
    for i in np.random.permutation(len(sentences_original)):
        tmp_sentences.append(sentences_original[i])
        tmp_next_word.append(next_original[i])

    cut_index = int(len(sentences_original) * (1.-(percentage_test/100.)))
    x_train, x_test = tmp_sentences[:cut_index], tmp_sentences[cut_index:]
    y_train, y_test = tmp_next_word[:cut_index], tmp_next_word[cut_index:]

    print("Size of training set = %d" % len(x_train))
    print("Size of test set = %d" % len(y_test))
    return (x_train, y_train), (x_test, y_test)


# Data generator for fit and evaluate
def generator(sentence_list, next_word_list, batch_size):
    index = 0
    while True:
        x = np.zeros((batch_size, SEQUENCE_LEN), dtype=np.int32)
        y = np.zeros((batch_size), dtype=np.int32)
        for i in range(batch_size):
            for t, w in enumerate(sentence_list[index % len(sentence_list)]):
                x[i, t] = word_indices[w]
            y[i] = word_indices[next_word_list[index % len(sentence_list)]]
            index = index + 1
        yield x, y


def get_model(dropout=0.2):
    print('Build model...')
    model = Sequential()
    model.add(Embedding(input_dim=len(words), output_dim=100))
    model.add(Bidirectional(LSTM(128)))
    if dropout > 0:
        model.add(Dropout(dropout))
    model.add(Dense(len(words)))
    model.add(Activation('softmax'))
    return model


# Functions from keras-team/keras/blob/master/examples/lstm_text_generation.py
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    examples_file.write('\n----- Generating text after Epoch: %d\n' % epoch)

    # Randomly pick a seed sequence
    seed_index = np.random.randint(len(sentences+sentences_test))
    seed = (sentences+sentences_test)[seed_index]

    for diversity in [0.3, 0.4, 0.5, 0.6, 0.7]:
        sentence = seed
        examples_file.write('----- Diversity:' + str(diversity) + '\n')
        examples_file.write('----- Generating with seed:\n"' + ' '.join(sentence) + '"\n')
        examples_file.write(' '.join(sentence))

        for i in range(50):
            x_pred = np.zeros((1, SEQUENCE_LEN))
            for t, word in enumerate(sentence):
                x_pred[0, t] = word_indices[word]

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_word = indices_word[next_index]

            sentence = sentence[1:]
            sentence.append(next_word)

            examples_file.write(" "+next_word)
        examples_file.write('\n')
    examples_file.write('='*80 + '\n')
    examples_file.flush()


if __name__ == "__main__":
    # Argument check
    examples = "./examples.txt"

    if not os.path.isdir('./checkpoints/'):
        os.makedirs('./checkpoints/')

    # with io.open(corpus, encoding='utf-8') as f:
    #     text = f.read().lower().replace('\n', ' \n ')
    # print('Corpus length in characters:', len(text))

    # text_in_words = [w for w in text.split(' ') if w.strip() != '' or w == '\n']
    # print('Corpus length in words:', len(text_in_words))
    text_in_words = []
    with open('/content/drive/My Drive/Augment4Gains/data/reddit/train.json') as f:
      data = json.load(f)
      for each_data in data:
        sentence = each_data['source']
        # sentences.append(sentence)
        processed_sentence = re.sub(r'\s+', ' ', sentence)
        words = word_tokenize(processed_sentence)
        text_in_words.extend(words)
    
    print('Corpus length in words:', len(text_in_words))
    # Calculate word frequency
    word_freq = {}
    for word in text_in_words:
        word_freq[word] = word_freq.get(word, 0) + 1

    ignored_words = set()
    for k, v in word_freq.items():
        if word_freq[k] < MIN_WORD_FREQUENCY:
            ignored_words.add(k)

    words = set(text_in_words)
    print('Unique words before ignoring:', len(words))
    print('Ignoring words with frequency <', MIN_WORD_FREQUENCY)
    words = sorted(set(words) - ignored_words)
    print('Unique words after ignoring:', len(words))

    word_indices = dict((c, i) for i, c in enumerate(words))
    indices_word = dict((i, c) for i, c in enumerate(words))

    # cut the text in semi-redundant sequences of SEQUENCE_LEN words
    sentences = []
    next_words = []
    ignored = 0
    for i in range(0, len(text_in_words) - SEQUENCE_LEN, STEP):
        # Only add the sequences where no word is in ignored_words
        if len(set(text_in_words[i: i+SEQUENCE_LEN+1]).intersection(ignored_words)) == 0:
            sentences.append(text_in_words[i: i + SEQUENCE_LEN])
            next_words.append(text_in_words[i + SEQUENCE_LEN])
        else:
            ignored = ignored + 1
    print('Ignored sequences:', ignored)
    print('Remaining sequences:', len(sentences))

    # x, y, x_test, y_test
    (sentences, next_words), (sentences_test, next_words_test) = shuffle_and_split_training_set(
        sentences, next_words
    )

    model = get_model()
    model.compile(loss='sparse_categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

    file_path = "./checkpoints/GENAUG-epoch{epoch:03d}-words%d-sequence%d-minfreq%d-" \
                "loss{loss:.4f}-accuracy{accuracy:.4f}-val_loss{val_loss:.4f}-val_accuracy{val_accuracy:.4f}" % \
                (len(words), SEQUENCE_LEN, MIN_WORD_FREQUENCY)

    checkpoint = ModelCheckpoint(file_path, monitor='val_accuracy', save_best_only=True)
    print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
    early_stopping = EarlyStopping(monitor='val_accuracy', patience=20)
    callbacks_list = [checkpoint, print_callback, early_stopping]

    examples_file = open(examples, "w")
    model.fit(generator(sentences, next_words, BATCH_SIZE),
                        steps_per_epoch=int(len(sentences)/BATCH_SIZE) + 1,
                        epochs=100,
                        callbacks=callbacks_list,
                        validation_data=generator(sentences_test, next_words_test, BATCH_SIZE),
                        validation_steps=int(len(sentences_test)/BATCH_SIZE) + 1)

Corpus length in words: 794259
Unique words before ignoring: 31458
Ignoring words with frequency < 10
Unique words after ignoring: 4773
Ignored sequences: 413886
Remaining sequences: 380363
Shuffling sentences
Size of training set = 372755
Size of test set = 7608
Build model...
Epoch 1/100
11649/11649 [==============================] - 124s 10ms/step - loss: 5.8625 - accuracy: 0.0934 - val_loss: 5.0585 - val_accuracy: 0.1633


INFO:tensorflow:Assets written to: ./checkpoints/GENAUG-epoch001-words4773-sequence10-minfreq10-loss5.4988-accuracy0.1272-val_loss5.0585-val_accuracy0.1633/assets


INFO:tensorflow:Assets written to: ./checkpoints/GENAUG-epoch001-words4773-sequence10-minfreq10-loss5.4988-accuracy0.1272-val_loss5.0585-val_accuracy0.1633/assets


Epoch 2/100
11649/11649 [==============================] - 122s 10ms/step - loss: 5.0010 - accuracy: 0.1694 - val_loss: 4.8942 - val_accuracy: 0.1787


INFO:tensorflow:Assets written to: ./checkpoints/GENAUG-epoch002-words4773-sequence10-minfreq10-loss4.9333-accuracy0.1748-val_loss4.8942-val_accuracy0.1787/assets


INFO:tensorflow:Assets written to: ./checkpoints/GENAUG-epoch002-words4773-sequence10-minfreq10-loss4.9333-accuracy0.1748-val_loss4.8942-val_accuracy0.1787/assets


Epoch 3/100
11649/11649 [==============================] - 123s 11ms/step - loss: 4.7487 - accuracy: 0.1857 - val_loss: 4.8410 - val_accuracy: 0.1843


INFO:tensorflow:Assets written to: ./checkpoints/GENAUG-epoch003-words4773-sequence10-minfreq10-loss4.7074-accuracy0.1887-val_loss4.8410-val_accuracy0.1843/assets


INFO:tensorflow:Assets written to: ./checkpoints/GENAUG-epoch003-words4773-sequence10-minfreq10-loss4.7074-accuracy0.1887-val_loss4.8410-val_accuracy0.1843/assets


Epoch 4/100
11649/11649 [==============================] - 125s 11ms/step - loss: 4.5672 - accuracy: 0.1972 - val_loss: 4.8493 - val_accuracy: 0.1879
